In [ ]:
import pandas as pd
import math
import numpy as np
import requests
from io import StringIO

pd.__version__

In [ ]:
def gitlab_raw_data(prj_id,prj_file,branch):
    '''
    read remote raw file from gitlab using api
    GET /projects/:id/repository/files/:file_path/raw
    file_path (required) - URL encoded full path to new file, such as lib%2Fclass%2Erb.
    ref (optional) - The name of branch, tag or commit. Default is the HEAD of the project.
    '''
    prj_id_q = requests.utils.quote(prj_id,safe='')
    prj_file_q = requests.utils.quote(prj_file,safe='')
    #data_url = "https://gitlab.advr.iit.it/api/v4/projects/hhmc-firmware%2Fmsp432-ft6/repository/files/calib%2Fsens_2%2Etxt/raw?ref=calib"
    data_url = f"https://gitlab.advr.iit.it/api/v4/projects/{prj_id_q}/repository/files/{prj_file_q}/raw?ref={branch}"
    print(data_url)
    headers = {"PRIVATE-TOKEN": "MCBxyjeT61Bngox7Ktas"}
    return StringIO(requests.get(data_url,headers=headers).text)

In [ ]:
#dL=0.044;  # position offset between sensors along z axis for WALKMAN feet
#dL=0.0346; # position offset between sensors along z axis for mini45/50 sensors 
#dL=0.1737; # New setup distance from from mini 45 ATI to the foot sensor.
#dL=0.065;  # New setup distance from from mini 45 ATI to the HERI hand sensor.
dL=0.159;  # New setup distance from from mini 45 ATI to the cogimon foot sensor.
#dL=0.0549; # New setup distance from from mini 45 ATI to the HERI hand sensor for Singapore.


In [ ]:
col_names =str.split("time_ns sg1 sg2 sg3 sg4 sg5 sg6 fx fy fz tx ty tz")
# read local file
data = pd.read_csv("sens.txt", sep="\t", names=col_names)
gitlab_file_arg = ("hhmc-firmware/msp432-ft6","calib/sens_1.txt","calib")
#data = pd.read_csv(gitlab_raw_data(*gitlab_file_arg), sep="\t", names=col_names)
#data = pd.DataFrame(np.arange(20).reshape(10,2))
data.info()

In [ ]:
clamp = 50 #1000
lower_half = data[clamp:math.ceil(len(data)/2)]
upper_half = data[math.ceil(len(data)/2):len(data)-clamp]
# Strain gauge raw data matrix (sg1, sg2, sg3, sg4, sg5, sg6) 
# calibration matrix identification
SGi = lower_half[col_names[1:7]].copy()
# calibration matrix verification
SGv = upper_half[col_names[1:7]].copy()
# ATI force torque data  matrix (fx,fy,fz tx,ty,tz)
# calibration matrix identification
FTi = lower_half[col_names[7:13]].copy()
# Map measured ATI Tx,Ty to our sensor frame at distance dL along z axis
# correct Tx torque for the dL    Tx_IIT = Tx_ATI-Fy_ATI*DL
FTi.tx -= FTi.fy*dL
# correct Ty torque for the dL    Ty_IIT = Ty_ATI+Fx_ATI*DL
FTi.ty += FTi.fx*dL
# calibration matrix verification
FTv = upper_half[col_names[7:13]].copy()
# correct Tx torque for the dL    Tx_IIT = Tx_ATI-Fy_ATI*DL
FTv.tx -= FTv.fy*dL
# correct Ty torque for the dL    Ty_IIT = Ty_ATI+Fx_ATI*DL
FTv.ty += FTv.fx*dL

In [ ]:
#SGi_inv = pd.DataFrame(np.linalg.pinv(SGi.values), SGi.columns, SGi.index)
#npSGi_inv = SGi_inv.to_numpy()

npSGi = SGi.to_numpy()
npSGi_inv = np.linalg.pinv(npSGi)
npFTi = FTi.to_numpy()
npCM = np.dot(npSGi_inv,npFTi)

In [ ]:
dfCM = pd.DataFrame(data=npCM.T.astype(float))
dfCM.to_csv('cal_mat.txt', sep=',', header=False, float_format='%.6f', index=False)

In [ ]:
npCM.T.tofile('cal_mat.bin',format='%f')
np.fromfile('cal_mat.bin')

In [ ]:
np.identity(6).tofile('ones.bin',format='%f')
np.fromfile('ones.bin')

In [ ]:
def write_mat_sig(np_mat, filename):
    with open(filename, "wb") as f:
        #f.write(bytearray([0xFE,0xCA,0x00,0x00])+np_mat.tobytes())
        #f.write(bytearray([0xFE,0xCA,0x00,0x00]))
        f.write(bytearray([0xFE,0xCA,0x0,0x0]))
        np_mat.tofile(f,format="%f")
    # test
    with open(filename, "rb") as f:
        np_back = np.fromfile(f,offset=4)

    return np_back        

ret = write_mat_sig(np.identity(6),"ones.bin")
print(ret)
ret = write_mat_sig(npCM.T,"cal_mat.bin")
print(ret)